In [29]:
import re
import torch
import numpy as np

from nltk.tokenize import RegexpTokenizer
from imblearn.over_sampling import RandomOverSampler
from google.colab import files
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from transformers import EarlyStoppingCallback, AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader

In [2]:
import pandas as pd
import io

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModel.from_pretrained("vinai/phobert-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [5]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Report1/train_data.csv')
val_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Report1/val_data.csv')

train_texts = train_df['content']
train_labels = train_df['label']
val_texts = val_df['content']
val_labels = val_df['label']

In [8]:
# Tokenize dữ liệu với padding
def tokenize_and_pad(texts, tokenizer, max_length=256):
    encodings = tokenizer(
        texts,
        truncation=True,
        padding='max_length',  # Thêm padding để tất cả các văn bản có cùng chiều dài
        max_length=max_length
    )
    return encodings

train_encodings = tokenize_and_pad(train_texts.tolist(), tokenizer, max_length=256)
val_encodings = tokenize_and_pad(val_texts.tolist(), tokenizer, max_length=256)

In [7]:
# Tokenize
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=256, truncation_strategy='longest_first')
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=256, truncation_strategy='longest_first')


In [9]:
# Kiểm tra dữ liệu sau khi token hóa
print(train_encodings.keys())
print(val_encodings.keys())
print(train_encodings['input_ids'][:2])  # Hiển thị một vài mẫu tokenized
print(val_encodings['input_ids'][:2])


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
[[0, 149, 5887, 9105, 16, 18, 45, 1500, 1088, 2398, 5130, 8138, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 1607, 6222, 47, 258, 482, 121, 1722, 1074, 17, 50, 604, 1410, 15, 9, 213, 193

In [10]:
train_labels = np.array(train_labels).astype(int)
val_labels = np.array(val_labels).astype(int)

In [11]:
# Kiểm tra nhãn
print(set(train_labels))
print(set(val_labels))


{0, 1}
{0, 1}


In [12]:
class FakeNewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).squeeze()  # Đảm bảo nhãn có kích thước đúng
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = FakeNewsDataset(train_encodings, train_labels.tolist())
val_dataset = FakeNewsDataset(val_encodings, val_labels.tolist())

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)


In [13]:
# Tải mô hình phân loại
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=2)

# Thiết lập các tham số huấn luyện
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Khởi tạo Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Huấn luyện mô hình
trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,0.706500
20,0.714500
30,0.696000
40,0.693400
50,0.687100
60,0.675000
70,0.667000
80,0.648900
90,0.615700
100,0.600200


TrainOutput(global_step=429, training_loss=0.4429106942979328, metrics={'train_runtime': 9759.135, 'train_samples_per_second': 0.351, 'train_steps_per_second': 0.044, 'total_flos': 450709237831680.0, 'train_loss': 0.4429106942979328, 'epoch': 3.0})

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

# Dự đoán nhãn cho tập kiểm tra
predictions = trainer.predict(val_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)

In [19]:
# Tính các chỉ số
accuracy = accuracy_score(val_labels, pred_labels)
precision = precision_score(val_labels, pred_labels, pos_label=0)
recall = recall_score(val_labels, pred_labels, pos_label=0)
f1 = f1_score(val_labels, pred_labels, pos_label=0)
auc = roc_auc_score(val_labels, predictions.predictions[:, 1])

print(f"Accuracy: {accuracy:.6f}")
print(f"Precision: {precision:.6f}")
print(f"Recall: {recall:.6f}")
print(f"F1 Score: {f1:.6f}")
print(f'AUC: {auc:.6f}')

Accuracy: 0.881119
Precision: 0.861111
Recall: 0.898551
F1 Score: 0.879433
AUC: 0.923423


In [20]:
accuracy = accuracy_score(val_labels, pred_labels)
precision = precision_score(val_labels, pred_labels, pos_label=1)
recall = recall_score(val_labels, pred_labels, pos_label=1)
f1 = f1_score(val_labels, pred_labels, pos_label=1)
auc = roc_auc_score(val_labels, predictions.predictions[:, 1])

print(f"Accuracy: {accuracy:.6f}")
print(f"Precision: {precision:.6f}")
print(f"Recall: {recall:.6f}")
print(f"F1 Score: {f1:.6f}")
print(f'AUC: {auc:.6f}')

Accuracy: 0.881119
Precision: 0.901408
Recall: 0.864865
F1 Score: 0.882759
AUC: 0.923423


In [21]:
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/bpe.codes',
 './saved_model/added_tokens.json')

In [22]:
output_dir = '/content/drive/MyDrive/Colab Notebooks/Report1/PhoBERT_fakenews'

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/Colab Notebooks/Report1/PhoBERT_fakenews/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Report1/PhoBERT_fakenews/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Report1/PhoBERT_fakenews/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/Report1/PhoBERT_fakenews/bpe.codes',
 '/content/drive/MyDrive/Colab Notebooks/Report1/PhoBERT_fakenews/added_tokens.json')

In [27]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained('./saved_model')
tokenizer = AutoTokenizer.from_pretrained('./saved_model')

inputs = tokenizer("Sập hầm thang Quảng Ninh con số thiệt mạng khổng lồ", return_tensors="pt")
# Thực hiện dự đoán
with torch.no_grad():
    outputs = model(**inputs)

# Lấy logits từ kết quả dự đoán
logits = outputs.logits
print(logits)

from torch.nn.functional import softmax

# Chuyển đổi logits thành xác suất
probs = softmax(logits, dim=1)
print(probs)

# Lấy lớp dự đoán (lớp có xác suất cao nhất)
predicted_class = torch.argmax(probs, dim=1)
print(predicted_class)


tensor([[ 0.5293, -0.3183]])
tensor([[0.7001, 0.2999]])
tensor([0])


In [31]:
# Thiết lập các tham số huấn luyện (có dừng sớm)
training_args_with_early_stop = TrainingArguments(
    output_dir='./results_with_early_stop',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs_with_early_stop',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    save_total_limit=3,
    load_best_model_at_end=True,
)

# Khởi tạo Trainer (có dừng sớm)
trainer_with_early_stop = Trainer(
    model=model,
    args=training_args_with_early_stop,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [32]:
trainer_with_early_stop.train()

Step,Training Loss,Validation Loss
50,0.272500,0.503063
100,0.116600,0.512094
150,0.138000,0.618269
200,0.118500,0.548692


TrainOutput(global_step=200, training_loss=0.11566500574350357, metrics={'train_runtime': 5732.6677, 'train_samples_per_second': 0.996, 'train_steps_per_second': 0.125, 'total_flos': 210225733232640.0, 'train_loss': 0.11566500574350357, 'epoch': 1.3986013986013985})

In [33]:
# Dự đoán nhãn cho tập kiểm tra
predictions2 = trainer_with_early_stop.predict(val_dataset)

# Lấy nhãn dự đoán từ logits
pred_labels2 = np.argmax(predictions2.predictions, axis=1)

# In ra một số dự đoán để kiểm tra
print(pred_labels2[:10])  # In ra 10 dự đoán đầu tiên

[0 1 0 0 1 0 0 1 1 0]


In [34]:
# Tính các chỉ số
accuracy2 = accuracy_score(val_labels, pred_labels2)
precision2 = precision_score(val_labels, pred_labels2, pos_label=0)
recall2 = recall_score(val_labels, pred_labels2, pos_label=0)
f12 = f1_score(val_labels, pred_labels2, pos_label=0)
auc2 = roc_auc_score(val_labels, predictions2.predictions[:, 1])

print(f"Accuracy: {accuracy2:.6f}")
print(f"Precision: {precision2:.6f}")
print(f"Recall: {recall2:.6f}")
print(f"F1 Score: {f12:.6f}")
print(f'AUC: {auc2:.6f}')

Accuracy: 0.874126
Precision: 0.835526
Recall: 0.920290
F1 Score: 0.875862
AUC: 0.927977
